# This is how a chain and chain flow looks like. 
### Chain is a wrapper around prompt and LLM. It takes input as dict and generated output as dict  
![title](pics/Screenshot_20240518_131454.png)

### Because it is a wrapper, the LLM model inside can be easily replaced in case of need
![title](pics/Screenshot_20240518_133026.png)

### Chains can be combined together so that the output from first is the input for second
### ![title](pics/Screenshot_20240518_131838.png)

### In order to make it work the names for OUTPUT/INPUT need to match
### ![title](pics/Screenshot_20240518_132152.png)

# Example of how to use instead on directly speaking to model

In [7]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

load_dotenv()
llm = OpenAI()

prompt = PromptTemplate(
    template = "What is the capital city of {state}",
    input_variables = ['state']
)

chain = LLMChain(
    llm=llm, 
    prompt=prompt
)

result = chain.invoke({"state": "Czech Republic"})
print(result)

{'state': 'Czech Republic', 'text': '\n\nThe capital city of Czech Republic is Prague.'}


 # Improved previous example where i put input variables into CLI and use pipelines of two chains
 ### this will not run inside jupyter, run it inside a file, eg *python file.py --state=USA*


In [12]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import argparse

load_dotenv()
llm = OpenAI()

parser = argparse.ArgumentParser()
parser.add_argument("--state", default="Czech republic")
args = parser.parse_args()

prompt = PromptTemplate(
    template = "What is the capital city of China",
    # input_variables = ['state']
)

chain = LLMChain(
    llm=llm, 
    prompt=prompt
)

result = chain.invoke({: args.state})
print(result)

usage: ipykernel_launcher.py [-h] [--state STATE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/nirvikalpa/.local/share/jupyter/runtime/kernel-7245ad86-2fa6-4a02-8f41-5e4e0006161d.json


SystemExit: 2

/home/nirvikalpa/.cache/pypoetry/virtualenvs/chatgptreporeader-BYS5M-4o-py3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


 # Improved previous example where i chain two chains together
### Chains can be combined together so that the output from first is the input for second
### ![title](pics/Screenshot_20240518_131838.png) 


In [23]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.sequential import SequentialChain
from dotenv import load_dotenv

load_dotenv()
llm = OpenAI()

# 1. prompt to generate code
code_prompt = PromptTemplate(
    template = "Write a function in {language} that does task: {task}",
    input_variables = ["language", "task"]
)
code_chain = LLMChain(
    llm=llm, 
    prompt=code_prompt, 
    output_key="code" # default is text, but here I am renaming it, the other two which came inside will also be returned
) 

# 2. prompt to generate test for the code
test_prompt = PromptTemplate(
    template="Generate test for this code {code} in language {language}",
    input_variables = ["code", "language"]  # these are the outputs from the first 'code_chain'
)
test_chain = LLMChain(
    llm=llm,
    prompt=test_prompt,
    output_key="test" # again renaming the default, the other two which came inside will also be returned
)

# here is where the magic happens - I need to wire the chains together 
# because it is a sequential action, 
chain = SequentialChain(
    chains=[code_chain, test_chain], # which chains to run
    input_variables=["language", "task"], # what goes into the very start
    output_variables=["code", "test"] # what is the final result + what came inside as always
)

result = chain.invoke({"language": "python", "task": "return a list of 5 numbers"})  # i am not using CLI so this is hardcoded here
print(result)

{'language': 'python', 'task': 'return a list of 5 numbers', 'code': '\n\ndef return_numbers():\n    return [1, 2, 3, 4, 5]', 'test': "\n\n\n# Importing necessary libraries\nimport unittest\n\n# Importing the function to be tested\nfrom return_numbers import return_numbers\n\n# Creating a test class\nclass TestReturnNumbers(unittest.TestCase):\n\n    # Defining a test method\n    def test_return_numbers(self):\n        \n        # Expected output\n        expected = [1, 2, 3, 4, 5]\n        \n        # Actual output\n        actual = return_numbers()\n        \n        # Asserting if the actual output is equal to the expected output\n        self.assertEqual(actual, expected)\n        \n# Allowing the test class to be executed directly        \nif __name__ == '__main__':\n    unittest.main()"}


In [31]:
print(result["code"])



def return_numbers():
    return [1, 2, 3, 4, 5]


In [34]:
print(result["test"])




# Importing necessary libraries
import unittest

# Importing the function to be tested
from return_numbers import return_numbers

# Creating a test class
class TestReturnNumbers(unittest.TestCase):

    # Defining a test method
    def test_return_numbers(self):
        
        # Expected output
        expected = [1, 2, 3, 4, 5]
        
        # Actual output
        actual = return_numbers()
        
        # Asserting if the actual output is equal to the expected output
        self.assertEqual(actual, expected)
        
# Allowing the test class to be executed directly        
if __name__ == '__main__':
    unittest.main()
